In [96]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import os
os.chdir('drive/MyDrive/dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

Reference: https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c

In [97]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import when

In [98]:
spark = pyspark.sql.SparkSession.builder.master('local[1]').appName('H1B').getOrCreate()
training_data = spark.read.csv('train.csv', header = True, inferSchema = True)

In [99]:
test_data = spark.read.csv('test.csv', header = True, inferSchema = True)

In [100]:
cols = training_data.columns
cols.remove('EMPLOYER_NAME')

Exception ignored in: <function JavaWrapper.__del__ at 0x7fcf88b437a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [101]:
training_data = training_data.drop(col('EMPLOYER_NAME'))
training_data = training_data.withColumn("CASE_DURATION",col("CASE_DURATION").cast("int"))
training_data = training_data.withColumn("FULL_TIME_POSITION",col("FULL_TIME_POSITION").cast("int"))
training_data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- AK1: integer (nullable = true)
 |-- AL2: integer (nullable = true)
 |-- AR3: integer (nullable = true)
 |-- AZ4: integer (nullable = true)
 |-- CA5: integer (nullable = true)
 |-- CO6: integer (nullable = true)
 |-- CT7: integer (nullable = true)
 |-- DC8: integer (nullable = true)
 |-- DE9: integer (nullable = true)
 |-- FL10: integer (nullable = true)
 |-- FM11: integer (nullable = true)
 |-- GA12: integer (nullable = true)
 |-- GU13: integer (nullable = true)
 |-- HI14: integer (nullable = true)
 |-- IA15: integer (nullable = true)
 |-- ID16: integer (nullable = true)
 |-- IL17: integer (nullable = true)
 |-- IN18: integer (nullable = true)
 |-- KS19: integer (nullable = true)
 |-- KY20: integer (nullable = true)
 |-- LA21: integer (nullable = true)
 |-- MA22: integer (nullable = true)
 |-- MD23: integer (nullable = true)
 |-- ME24: integer (nullable = true)
 |-- MH: integer (nullable = true)
 |-- MI26: integer (nullable = true)
 |-- MN2

In [102]:
from pyspark.ml.feature import VectorAssembler
try:
  cols.remove('CASE_STATUS')
except:
  pass
cols.remove('_c0')
training_data = training_data.drop(col('_c0'))
assembler = VectorAssembler(inputCols=cols,outputCol="features", handleInvalid='skip')

In [103]:
training_data=assembler.transform(training_data)

In [104]:
training_data.select("features").show()

+--------------------+
|            features|
+--------------------+
|(756,[42,64,159,2...|
|(756,[4,121,159,1...|
|(756,[49,92,159,2...|
|(756,[7,64,159,17...|
|(756,[38,62,159,2...|
|(756,[4,64,159,16...|
|(756,[4,60,159,16...|
|(756,[4,64,159,16...|
|(756,[49,64,159,1...|
|(756,[39,81,159,1...|
|(756,[9,62,159,17...|
|(756,[54,65,159,2...|
|(756,[4,64,159,16...|
|(756,[4,65,159,21...|
|(756,[42,64,159,1...|
|(756,[26,64,159,1...|
|(756,[49,59,159,2...|
|(756,[11,64,159,2...|
|(756,[33,64,159,1...|
|(756,[31,64,159,2...|
+--------------------+
only showing top 20 rows



In [105]:
training_data = training_data.withColumn("CASE_STATUS",
                                         when(training_data.CASE_STATUS == 'CERTIFIED',1)
                                         .otherwise(when(training_data.CASE_STATUS == 'DENIED',0)
                                         .otherwise(when(training_data.CASE_STATUS == 'WITHDRAWN',3)
                                         .otherwise(2))))


In [106]:
from pyspark.ml.classification import LogisticRegression

In [107]:
lr = LogisticRegression(labelCol = 'CASE_STATUS',
                        featuresCol = 'features',
                        maxIter=10, regParam=0, elasticNetParam=0.8).setFamily("multinomial")

In [108]:
lr.fit(training_data)

LogisticRegressionModel: uid=LogisticRegression_8c18e54108c2, numClasses=4, numFeatures=756

In [64]:
from pyspark.sql.functions import isnan, count
training_data.select([count(when(isnan(c), c)).alias(c) for c in cols]).show()

+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------------+----+-----------+---------------+-----+-----+-----+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

In [110]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [111]:
# TBC